In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from utils import mnist, plot_graphs, plot_mnist, to_onehot
import numpy as np
import os 

%matplotlib inline

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
mnist_tanh = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,)),
                lambda x: x.to(device)
           ])

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

lr = 0.0001
prior_size = 10
train_epoch = 200
batch_size = 50

train_loader, valid_loader, test_loader = mnist(batch_size=batch_size, valid=10000, transform=mnist_tanh)

cuda:0


In [5]:
class FullyConnected(nn.Module):
    def __init__(self, sizes, dropout=False, activation_fn=nn.Tanh(), flatten=False, 
                 last_fn=None, first_fn=None, device='cpu'):
        super(FullyConnected, self).__init__()
        layers = []
        self.flatten = flatten
        if first_fn is not None:
            layers.append(first_fn)
        for i in range(len(sizes) - 2):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            if dropout:
                layers.append(nn.Dropout(dropout))
            layers.append(activation_fn) # нам не нужен дропаут и фнкция активации в последнем слое
        else: 
            layers.append(nn.Linear(sizes[-2], sizes[-1]))
        if last_fn is not None:
            layers.append(last_fn)
        self.model = nn.Sequential(*layers)
        self.to(device)
        
    def forward(self, x):
        if self.flatten:
            x = x.view(x.shape[0], -1)
        out = self.model(x)
        return out

In [6]:
class Classifier():
    def __init__(self, train_loader, valid_loader):
        self.net = FullyConnected([28*28, 1024, 1024, prior_size], activation_fn=nn.ReLU(), flatten=True, device=device)
        self.optimizer = optim.Adam(self.net.parameters(), lr=lr)
        self.criterion = nn.CrossEntropyLoss()
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.loss = 0.0
        self.loss_log = []
        self.acc_log = []
        
    def train(self, epoch):
        train_size = len(self.train_loader.sampler)
        print (train_size)
        for batch_idx, (data, label) in enumerate(self.train_loader):
            label = label.to(device)
            
            # train
            self.optimizer.zero_grad()              
            output = self.net(data)       
            self.loss = criterion(output, label)

            self.loss.backward()
            self.optimizer.step()

            if batch_idx % 200 == 0:
                line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
                    epoch, batch_idx * len(data), train_size, 100. * batch_idx / len(self.train_loader))
                losses = '{:.4f}'.format(self.loss.item())
                print(line + losses)

        else:
            batch_idx += 1
            line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
                epoch, batch_idx * len(data), train_size, 100. * batch_idx / len(self.train_loader))
            losses = '{:.4f}'.format(self.loss.item())
            print(line + losses)            
        
    def test(self, epoch):
        test_size = len(self.valid_loader)
        loss = 0.
        total = 0
        correct = 0        
        with torch.no_grad():
            for data, label in self.valid_loader:
                label = label.to(device)
                output = self.net(data)
                loss += self.criterion(output, label)

                _, predicted = torch.max(output.data, 1)
                total += label.size(0)
                correct += (predicted == label).sum().item()

            loss /= test_size       
            
        accuracy = 100 * correct / total

        report = 'Test loss: {:.4f} Accuracy: {:.2f} \n'.format(loss, accuracy)
        print(report)    
        
        self.loss_log.append(loss.item())
        self.acc_log.append(accuracy)

    def start_training(self, train_epoch):
        for epoch in range(1, train_epoch + 1):
            self.net.train()
            self.train(epoch)
            self.net.eval()   
            self.test(epoch)


### Классификатор на исходных данных

In [7]:
cls_raw = Classifier(train_loader, valid_loader)
cls_raw.start_training(10)

50000


NameError: name 'criterion' is not defined

In [32]:
print (cls_raw.loss_log)
print (cls_raw.acc_log)

[0.20352044701576233, 0.136849507689476, 0.1174914613366127, 0.09716933965682983, 0.08570578694343567, 0.08068057894706726, 0.0839589461684227, 0.07316433638334274, 0.09270811080932617, 0.0617312416434288]
[94.07, 95.73, 96.27, 97.01, 97.24, 97.58, 97.38, 97.7, 97.09, 98.12]


In [72]:
class Custom_dataset():
    def __init__(self, ptype='train', path=''):
        if ptype == 'train':
            self.train_data = torch.load(path)
        if ptype == 'valid':
            self.valid_data = torch.load(path)
        if ptype == 'test':
            self.test_data = torch.load(path)


In [74]:
caae_train = Custom_dataset('train', './caae_train_data')
#        caae_dataset.valid_data = torch.load('./caae_valid_data')
#        caae_datatet.test_data = torch.load('./caae_test_data')


In [76]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

caae_train_indices = list(range(len(caae_train_data[0])))
caae_valid_indices = list(range(len(caae_valid_data[0])))
print ('ind=', len(caae_train_indices))


caae_train_loader = DataLoader(caae_train, batch_size=250, sampler=SubsetRandomSampler(caae_train_indices))
#caae_valid_loader = DataLoader(caae_valid_data, batch_size=250, sampler=SubsetRandomSampler(caae_valid_indices))

ind= 50000


In [78]:
for t in caae_train:
    print(t)

TypeError: 'Custom_dataset' object is not iterable

In [30]:
cls_raw = Classifier(caae_train_loader, caae_valid_loader)
cls_raw.start_training(10)

50000


IndexError: tuple index out of range

In [63]:
def mnist1(batch_size=50, valid=0, shuffle=True, transform=mnist_tanh, path='./MNIST_data'):
    test_data = datasets.MNIST(path, train=False, download=True, transform=transform)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    
    train_data = datasets.MNIST(path, train=True, download=True, transform=transform)
    if valid > 0:
        num_train = len(train_data)
        print ('num=', num_train)
        indices = list(range(num_train))
        split = num_train-valid
        np.random.shuffle(indices)
        print ('ind=', len(indices))
        
        print('train_type=', type(train_data.train_data))
        print ('train=', train_data.train_data.shape)

        train_idx, valid_idx = indices[:split], indices[split:]
        train_sampler = SubsetRandomSampler(train_idx)
        valid_sampler = SubsetRandomSampler(valid_idx)

        train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
        valid_loader = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
    
        return train_loader, valid_loader, test_loader
    else:
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=shuffle)
        return train_loader, test_loader

In [64]:
train1_loader, valid1_loader, test1_loader = mnist1(batch_size=batch_size, valid=10000, transform=mnist_tanh)

num= 60000
ind= 60000
train_type= <class 'torch.Tensor'>
train= torch.Size([60000, 28, 28])
